In [1]:
%matplotlib inline

import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import fiona
from pyproj import Proj, transform

In [2]:
conn = psycopg2.connect(dbname="gis", user="postgres", password="")

In [3]:
rapperswil_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1683921"

In [4]:
pubs_rapperswil = gpd.read_postgis(
    "SELECT * FROM planet_osm_point WHERE amenity = 'pub' AND st_within(way, ({}))".format(rapperswil_polygon_query), 
    conn, geom_col='way', crs='epsg:3857')

In [5]:
pubs_rapperswil

,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,tourism,tower:type,tunnel,water,waterway,wetland,width,wood,z_order,way
0,2045478751,None,None,1,None,None,None,None,pub,None,...,None,None,None,None,None,None,None,None,None,POINT (981440.51 5978942.61)
1,1355213714,None,None,9,None,None,None,None,pub,None,...,None,None,None,None,None,None,None,None,None,POINT (981344.77 5979063.34)
2,2045478750,None,None,None,None,None,None,None,pub,None,...,None,None,None,None,None,None,None,None,None,POINT (981499.6 5978969.41)
3,291417612,None,None,None,None,None,None,None,pub,None,...,None,None,None,None,None,None,None,None,None,POINT (985378.59 5978891.86)


In [6]:
m = folium.Map(location=[47.2271, 8.8245], zoom_start=15)
inProj = Proj(init=pubs_rapperswil.crs)
outProj = Proj(init='epsg:4326')

for index, pub in pubs_rapperswil.iterrows():
    x,y = pub['way'].x, pub['way'].y
    x, y = transform(inProj, outProj, x, y)
    folium.Marker([y, x], popup=pub['name']).add_to(m)
    
m

In [7]:
rapperswil_pubs_polygons = gpd.read_postgis(
    """SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
        INNER JOIN planet_osm_point AS point
            ON st_within(point.way, polygon.way)
        WHERE point.amenity = 'pub'
            AND st_within(point.way, ({}))
            AND polygon.building = 'yes'""".format(rapperswil_polygon_query), 
    conn, geom_col='geometry')

rapperswil_pubs_polygons

,geometry
0,"POLYGON ((981429.12 5978936.74, 981429.75 5978..."
1,"POLYGON ((981326.96 5979071.09, 981329.67 5979..."
2,"POLYGON ((981454.26 5978958.53, 981470.8100000..."
3,"POLYGON ((985368.0600000001 5978903.55, 985414..."


In [8]:
rapperswil_pubs_polygons.crs = fiona.crs.from_epsg(3857)

In [9]:
m = folium.Map(location=[47.226, 8.818], zoom_start=17)

folium.GeoJson(rapperswil_pubs_polygons).add_to(m)

m

In [10]:
rapperswil_poi_polygons = gpd.read_postgis(
    """SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
        INNER JOIN planet_osm_point AS point
            ON st_within(point.way, polygon.way)
        WHERE point.amenity IN ('pub', 'restaurant')
            AND st_within(point.way, ({}))
            AND polygon.building = 'yes'""".format(rapperswil_polygon_query), 
    conn, geom_col='geometry')

rapperswil_poi_polygons.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=[47.226, 8.818], zoom_start=17)

folium.GeoJson(rapperswil_poi_polygons).add_to(m)

m